In [2]:
epub_path = '../../Resources/مجنون ليلى.epub'
book_title = 'مجنون ليلى'
example_xhtml = "chapter-1-2-L.xhtml"

## This function is general for any EPUB file

In [13]:
import zipfile
from bs4 import BeautifulSoup
import os

def extract_and_save_chapters(epub_path, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    with zipfile.ZipFile(epub_path, 'r') as zip_ref:
        zip_ref.extractall("temp_epub_extraction")

    chapter_counter = 1
    for root, dirs, files in sorted(os.walk("temp_epub_extraction")):
        for file in sorted(files):
            if file.endswith('.html') or file.endswith('.xhtml'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    soup = BeautifulSoup(f, 'lxml')
                    texts = soup.get_text(separator='\n', strip=True)
                    chapter_filename = f"chapter_{chapter_counter}.txt"
                    chapter_path = os.path.join(output_directory, chapter_filename)
                    with open(chapter_path, 'w', encoding='utf-8') as chapter_file:
                        chapter_file.write(texts)
                    chapter_counter += 1
    
    # Cleanup extraction directory
    for root, dirs, files in os.walk("temp_epub_extraction", topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir("temp_epub_extraction")

# Specify the directory where you want to save the chapters
output_directory = f'./{book_title}_chapters'

extract_and_save_chapters(epub_path, output_directory)

In [10]:
from lxml import etree

# Load the XHTML content from the file
with open(example_xhtml, "rb") as file:
    # Parse the XHTML
    tree = etree.parse(file)


# Use list comprehension to get all elements
    ## it checks if the element has text and if the text is not empty (whitespace)
elements_list = [element for element in tree.iter() if element.text and
                element.text.strip() and
                element.text.strip() != ':' and
                element.tag != '{http://www.w3.org/1999/xhtml}a']

In [13]:
# Extract the text from elements in elements_list if their tag is {http://www.w3.org/1999/xhtml}b, and store it in a list
characters_names = [element.text for element in elements_list if element.tag == '{http://www.w3.org/1999/xhtml}b']
# Make a set of the characters names to remove duplicates
characters_names = list(set(characters_names))

for name in characters_names:
    print(name)

هند
زياد
سعد
عبلة
ليلى
أصوات
سلمى
المهدي
بشر
منازل
عفراء
قيس
ابن ذريح
فتاة


In [14]:
# Extract the text from elements in elements_list if their text starts with ( and ends with ), and store it in a list
dialogues = [element.text for element in elements_list if element.text.startswith("(") and element.text.endswith(")")]

for dialogue in dialogues:
    print(dialogue)


(ساحة أمام خيام المهدي في حي بني عامر - مجلس من مجالس السمر في هذه
            الساحة - فتية وفتيات من الحي يسمرون في أوائل الليل، وفي أيدي الفتيات صوف ومغازل يلهون
            بها وهم يتحدثون — تخرج ليلى من خيام أبيها عند ارتفاع الستار ويدها في يد ابن ذريح)
(تصافحه سلمى)
(تصافحه هند ويحتفي به السامرون)
(تتضاحك الفتيات وتقول إحداهن لأخرى)
(تحاول ليلى أن تمد رجلها فتتألم وتستغيث)
(لنفسها)
(يبدو على ليلى شيء من الزهو فتتهامس الفتيات)
(فإنك لي جارٌ ولا ترهبِ الدَّهْرَا)
(ضحك من الجميع)
(ثم تقول في لوعة وصوت مخفوض وكأنما تحدث نفسها)
(تسترسل في حديثها الأول)
(تهيب بالسامرين وقد بلغ بها الغضب أقصاه)
(تدخل خباءها بينما ينفض السامرون فلا يتثاقل منهم في القيام إلا
                منازل — الهرج والأسف يسودان الجميع)
(تسمع ضحكاتهم من أقصى الطريق بينما يظهر قيس وزياد من جانب المسرح
                الآخر)
(يتقدم منهما خطوات)
(يجره إلى حيث تسمع أصواتهما من بعيد ثم تختفي فيقبل قيس على خباء
                ليلى وينادي)
(تظهر ليلى على باب الخباء)
(تخرج عفراء وتتبعها ليلى)
(تدخل ليلى)
(يترنح قيس في موقف

In [20]:
speakers = characters_names
speakers.append('المعلق')

current_speaker = 'المعلق'

# A list of tuples that will store the speaker and the dialogue, it should have the same length as elements_list
dialogues_and_speakers = []

for element in elements_list:
    # eliminate the whitespace from the start and end of the text
    element.text = element.text.strip()

    # if the element's tag is {http://www.w3.org/1999/xhtml}b, then the current_speaker should be the text of the element
    if element.tag == '{http://www.w3.org/1999/xhtml}b':
        # Check if the current_speaker is in the list of speakers
        if element.text in speakers:
            dialogues_and_speakers.append((element.text, "المعلق"))
            current_speaker = element.text
            continue
        else:
            current_speaker = 'المعلق'
    if element.text.startswith("(") and element.text.endswith(")"):
        dialogues_and_speakers.append((element.text, "المعلق"))
    else:
        dialogues_and_speakers.append((element.text, current_speaker))

for dialogue, speaker in dialogues_and_speakers:
    print(f"{speaker}: {dialogue}")
    print()


المعلق: مجنون ليلى

المعلق: الفصل الأول

المعلق: (ساحة أمام خيام المهدي في حي بني عامر - مجلس من مجالس السمر في هذه
            الساحة - فتية وفتيات من الحي يسمرون في أوائل الليل، وفي أيدي الفتيات صوف ومغازل يلهون
            بها وهم يتحدثون — تخرج ليلى من خيام أبيها عند ارتفاع الستار ويدها في يد ابن ذريح)

المعلق: ليلى

ليلى: دعي الغزْلَ سلمى وحَيِّي معي

ليلى: منارَ الحِجَازِ فَتَى يَثْرِبِ

المعلق: (تصافحه سلمى)

ليلى: ويا هِنْدُ هذا أديبُ الحِجازِ

ليلى: هلمِّي بمَقْدَمِهِ رَحِّبِي

المعلق: (تصافحه هند ويحتفي به السامرون)

المعلق: سعد

سعد: أمن يثربٍ أنت آتٍ؟

المعلق: ابن ذريح

ابن ذريح: أجل

ابن ذريح: من البلدِ القُدُس الطيِّب

المعلق: ليلى

ليلى: أيابنَ ذَريحٍ لقينا الغمام

المعلق: هند

هند: وطَافتْ بنا نَفَحَاتُ النبي

المعلق: عبلة

المعلق: (هامسة إلى سعد)

عبلة: مَن ابْنُ ذَريحٍ؟

المعلق: سعد

سعد: فتًى ذِكرُه

سعد: على مَشرِق الشمس والمغرب

سعد: رَضيعُ الحُسَيْنِ عليه السلامُ

سعد: وترْبُ الحُسَيْنِ من المكتبِ

المعلق: عبلة

المعلق: (إلى بشر ومشير إلى ابن ذريح)

عبلة: أتسمَعُ 

In [22]:
speakers = characters_names  # Presumed to be defined earlier
speakers.append('المعلق')

current_speaker = 'المعلق'

# A list of tuples that will store the speaker and the dialogue
dialogues_and_speakers = []

for element in elements_list:
    cleaned_text = element.text.strip()  # Clean up the text for processing

    # Check if the element's tag indicates a speaker
    if element.tag == '{http://www.w3.org/1999/xhtml}b':
        if cleaned_text in speakers:
            # Announce the speaker's name by the narrator before the speaker's dialogue
            dialogues_and_speakers.append((cleaned_text, "المعلق"))
            current_speaker = cleaned_text  # Update the current speaker
        else:
            current_speaker = 'المعلق'  # Reset to narrator if the speaker is not recognized
    elif cleaned_text.startswith("(") and cleaned_text.endswith(")"):
        # Special handling for narrative text
        dialogues_and_speakers.append((cleaned_text, "المعلق"))
    else:
        # Normal dialogue
        dialogues_and_speakers.append((cleaned_text, current_speaker))

# Printing dialogues and their speakers
for dialogue, speaker in dialogues_and_speakers:
    print(f"{speaker}: {dialogue}\n")


المعلق: مجنون ليلى

المعلق: الفصل الأول

المعلق: (ساحة أمام خيام المهدي في حي بني عامر - مجلس من مجالس السمر في هذه
            الساحة - فتية وفتيات من الحي يسمرون في أوائل الليل، وفي أيدي الفتيات صوف ومغازل يلهون
            بها وهم يتحدثون — تخرج ليلى من خيام أبيها عند ارتفاع الستار ويدها في يد ابن ذريح)

المعلق: ليلى

ليلى: دعي الغزْلَ سلمى وحَيِّي معي

ليلى: منارَ الحِجَازِ فَتَى يَثْرِبِ

المعلق: (تصافحه سلمى)

ليلى: ويا هِنْدُ هذا أديبُ الحِجازِ

ليلى: هلمِّي بمَقْدَمِهِ رَحِّبِي

المعلق: (تصافحه هند ويحتفي به السامرون)

المعلق: سعد

سعد: أمن يثربٍ أنت آتٍ؟

المعلق: ابن ذريح

ابن ذريح: أجل

ابن ذريح: من البلدِ القُدُس الطيِّب

المعلق: ليلى

ليلى: أيابنَ ذَريحٍ لقينا الغمام

المعلق: هند

هند: وطَافتْ بنا نَفَحَاتُ النبي

المعلق: عبلة

المعلق: (هامسة إلى سعد)

عبلة: مَن ابْنُ ذَريحٍ؟

المعلق: سعد

سعد: فتًى ذِكرُه

سعد: على مَشرِق الشمس والمغرب

سعد: رَضيعُ الحُسَيْنِ عليه السلامُ

سعد: وترْبُ الحُسَيْنِ من المكتبِ

المعلق: عبلة

المعلق: (إلى بشر ومشير إلى ابن ذريح)

عبلة: أتسمَعُ 

In [9]:
print(len(elements_list))
for i, element in enumerate(elements_list):
    assert isinstance(element, etree._Element)
    print(f"Element {i + 1}")
    print("-------------------------")
    print(element.tag)
    print(type(element.text))
    print(element.text)
    
    print("-------------------------")
    print()

    # The same output, but instead of the console, it is saved to a file called test.txt
    with open("test.txt", "a", encoding='utf-8') as file:
        file.write(f"Element {i + 1}\n")
        file.write("-------------------------\n")
        file.write(element.tag + "\n")
        file.write(str(type(element.text)) + "\n")
        file.write(element.text + "\n")
        file.write("-------------------------\n\n")


660
Element 1
-------------------------
{http://www.w3.org/1999/xhtml}title
<class 'str'>
 مجنون ليلى
-------------------------

Element 2
-------------------------
{http://www.w3.org/1999/xhtml}h1
<class 'str'>
الفصل الأول
-------------------------

Element 3
-------------------------
{http://www.w3.org/1999/xhtml}p
<class 'str'>
(ساحة أمام خيام المهدي في حي بني عامر - مجلس من مجالس السمر في هذه
            الساحة - فتية وفتيات من الحي يسمرون في أوائل الليل، وفي أيدي الفتيات صوف ومغازل يلهون
            بها وهم يتحدثون — تخرج ليلى من خيام أبيها عند ارتفاع الستار ويدها في يد ابن ذريح)
-------------------------

Element 4
-------------------------
{http://www.w3.org/1999/xhtml}b
<class 'str'>
ليلى
-------------------------

Element 5
-------------------------
{http://www.w3.org/1999/xhtml}div
<class 'str'>
دعي الغزْلَ سلمى وحَيِّي معي
-------------------------

Element 6
-------------------------
{http://www.w3.org/1999/xhtml}div
<class 'str'>
منارَ الحِجَازِ فَتَى يَثْرِبِ
------------